In [0]:
silver_stream = spark.readStream.format('delta').load(silver_path)

agg = (silver_stream
  .groupBy(F.window('event_ts','5 minutes'), F.col('customer_id'))
  .agg(F.sum('amount').alias('revenue'), F.count('*').alias('orders'))
  .select(F.col('window.start').alias('window_start'),
          F.col('window.end').alias('window_end'),
          'customer_id','revenue','orders'))

gold_path = 'dbfs:/pipelines/orders/gold_revenue'
ckpt_gold = 'dbfs:/pipelines/orders/_ckpt_gold'

(agg.writeStream
  .format('delta')
  .outputMode('complete')
  .option('checkpointLocation', ckpt_gold)
  .trigger(processingTime='1 minute')
  .start(gold_path))